## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-26-09-00-00 +0000,wsj,Trade Remains ‘Challenging’ for Eurozone Amid ...,https://www.wsj.com/articles/trade-remains-cha...
1,2026-02-26-09-00-00 +0000,cbc,Carney's India trip is all business: An inside...,https://www.cbc.ca/news/politics/carney-india-...
2,2026-02-26-09-00-00 +0000,cbc,Ottawa commits $1.55B to Jordan's Principle,https://www.cbc.ca/news/politics/jordans-princ...
3,2026-02-26-08-51-00 +0000,wsj,"Gold Slips but Remains Above $5,100 as Markets...",https://www.wsj.com/finance/commodities-future...
4,2026-02-26-08-49-58 +0000,nypost,Near-blind refugee found dead in Buffalo after...,https://nypost.com/2026/02/26/us-news/near-bli...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2615/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
54,trump,73
46,union,25
27,iran,19
362,epstein,14
713,speech,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
6,2026-02-26-08-35-46 +0000,nypost,Rep. Ilhan Omar claims State of the Union gues...,https://nypost.com/2026/02/26/us-news/rep-ilha...,141
135,2026-02-25-23-07-26 +0000,startribune,Ilhan Omar’s State of the Union guest arrested...,https://www.startribune.com/ilhan-omars-state-...,140
108,2026-02-26-00-26-00 +0000,wsj,Here’s a look at whether the math works on som...,https://www.wsj.com/politics/policy/the-math-b...,139
243,2026-02-25-18-29-00 +0000,wsj,"Casey Means, President Trump’s surgeon-general...",https://www.wsj.com/politics/policy/surgeon-ge...,136
298,2026-02-25-15-13-00 +0000,wsj,Didn’t catch all of President Trump’s State of...,https://www.wsj.com/politics/policy/trump-stat...,136


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
6,141,2026-02-26-08-35-46 +0000,nypost,Rep. Ilhan Omar claims State of the Union gues...,https://nypost.com/2026/02/26/us-news/rep-ilha...
161,62,2026-02-25-22-28-00 +0000,wsj,Spy agency blocks Congress from seeing Gabbard...,https://www.wsj.com/politics/policy/spy-agency...
243,57,2026-02-25-18-29-00 +0000,wsj,"Casey Means, President Trump’s surgeon-general...",https://www.wsj.com/politics/policy/surgeon-ge...
70,48,2026-02-26-03-02-00 +0000,wsj,The Trump administration is pausing nearly $26...,https://www.wsj.com/politics/policy/vance-anno...
46,46,2026-02-26-05-00-07 +0000,nyt,A Deal or War? Crucial Talks to Begin Between ...,https://www.nytimes.com/2026/02/26/world/middl...
256,45,2026-02-25-17-17-41 +0000,latimes,FBI raids of LAUSD Supt. Alberto Carvalho's ho...,https://www.latimes.com/california/story/2026-...
221,41,2026-02-25-19-43-12 +0000,nypost,Kathy Hochul demands $13.5B in tariff refunds ...,https://nypost.com/2026/02/25/business/ny-gov-...
184,37,2026-02-25-21-34-00 +0000,wsj,President Trump has ratcheted up pressure on S...,https://www.wsj.com/politics/policy/trumps-pus...
366,36,2026-02-25-09-56-55 +0000,cbc,Any trade deal with Canada will include tariff...,https://www.cbc.ca/news/politics/trump-tariffs...
108,35,2026-02-26-00-26-00 +0000,wsj,Here’s a look at whether the math works on som...,https://www.wsj.com/politics/policy/the-math-b...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
